In [1]:
import json

In [2]:
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import torch.optim as optim
import random
import torch.nn.functional as F

In [3]:
path1 = '../manning/16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json'
path2 = '../manning/16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json'

In [4]:
# this dictionary  provided by the tutor
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [5]:
def extractvalues(path):
    data = [json.loads(line) for line in open(path, 'r')]

    for d in data:
        dataset.append([d][0]["text"])
        target.append([d][0]["title"])


In [6]:
stop_words = stopwords.words('english')
# this function is provided by the tutor
def preprocess(text):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text

In [7]:
dataset = []
target = []
extractvalues(path1)
extractvalues(path2)

In [8]:
len(dataset)

94403

In [9]:
len(target)

94403

In [10]:
X = []
Y = []
for item in dataset:
    X.append(preprocess(item))

for item in target:
    Y.append(preprocess(item))

In [11]:
len(X)

94403

In [12]:
dataset[0]

'Chinese health authorities confirm 44 cases of mystery viral pneumonia in Wuhan India Blooms News Service | @indiablooms | 03 Jan 2020, 08:26 pm #China , #ViralPneumonia , #Wuhan\nBeijing/Sputnik: The toll of people infected with pneumonia of unknown kind in the Wuhan city of the central Chinese province of Hubei has risen to 44 as authorities fear the return of deadly Severe Acute Respiratory Syndrome (SARS) coronavirus, the local health committee said in a statement on Friday.\nThe Wuhan authorities confirmed the outbreak on Dec 31, saying that 27 people have tested positive since the beginning of that month.\n"As of January 3, a total of 44 pneumonia cases have been detected, with 11 people currently in severe condition, while the vital signs of the rest are considered generally stable," the Wuhan health authorities said.\nThe infected patients have been quarantined along with 121 people who had been in close contact with them, the statement read.\nAn investigation of experts from 

In [13]:
preprocess(dataset[0])

'chinese health authorities confirm 44 cases mystery viral pneumonia wuhan india blooms news service  indiablooms  03 jan 2020 0826 pm china  viralpneumonia  wuhan beijingsputnik toll people infected pneumonia unknown kind wuhan city central chinese province hubei risen 44 authorities fear return deadly severe acute respiratory syndrome  coronavirus local health committee said statement friday .  wuhan authorities confirmed outbreak dec 31 saying 27 people tested positive since beginning month .  as january 3 total 44 pneumonia cases detected 11 people currently severe condition vital signs rest considered generally stable wuhan health authorities said .  infected patients quarantined along 121 people close contact them statement read .  investigation experts central chinese national health commission found outbreak wuhan could begun seafood market .  time nature viral outbreak remains unknown .  investigators far ruled common flu avian flu adenovirus infection common respiratory disea

In [14]:
short_text=[]
short_summary=[]
max_len_text = 600
max_len_target = 30

for i in range(len(dataset)):
    if(len(target[i].split())<=max_len_target and len(dataset[i].split())<=max_len_text):
        short_text.append(dataset[i])
        short_summary.append(target[i])

temp_df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [15]:
len(temp_df)

64893

In [16]:
len(temp_df['text'])

64893

In [17]:
len(temp_df['summary'])

64893

In [18]:
temp_df.head()

,text,summary
0,Chinese health authorities confirm 44 cases of...,Chinese health authorities confirm 44 cases of...
1,The following is issued on behalf of the Hospi...,Update on cluster of patients infected with Co...
2,Beijing (AFP)\nChina on Friday confirmed more...,China confirms more cases of mystery viral pne...
3,China confirms more cases of mystery viral pne...,China confirms more cases of mystery viral pne...
4,The Centers for Disease Control (CDC) yesterda...,"Outbreak in China: Avian flu, adenovirus not c..."


In [19]:
# temp_df.isnull().values.any()

In [20]:
# temp_df.isnull().sum()

In [21]:
newdf = temp_df[temp_df['summary'].str.strip().astype(bool)]

In [22]:
len(newdf)

62633

In [23]:
df = newdf[newdf['text'].str.strip().astype(bool)]

In [24]:
len(df)

62358

In [25]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [26]:
def readData(text, summary):
    print("Reading lines...")
    
    # Split every line into pairs and normalize
    pairs = [[text[i],summary[i]] for i in range(len(text))]

    
    input_lang = Lang(text)
    output_lang = Lang(summary)

    return input_lang, output_lang, pairs
    
def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readData(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    return input_lang, output_lang, pairs

In [27]:
in_lang, out_lang, pairs = prepareData(list(df['text']), list(df['summary']))

Reading lines...
Read 62358 sentence pairs
Counting words...


In [28]:
pairs

[['Chinese health authorities confirm 44 cases of mystery viral pneumonia in Wuhan India Blooms News Service | @indiablooms | 03 Jan 2020, 08:26 pm #China , #ViralPneumonia , #Wuhan\nBeijing/Sputnik: The toll of people infected with pneumonia of unknown kind in the Wuhan city of the central Chinese province of Hubei has risen to 44 as authorities fear the return of deadly Severe Acute Respiratory Syndrome (SARS) coronavirus, the local health committee said in a statement on Friday.\nThe Wuhan authorities confirmed the outbreak on Dec 31, saying that 27 people have tested positive since the beginning of that month.\n"As of January 3, a total of 44 pneumonia cases have been detected, with 11 people currently in severe condition, while the vital signs of the rest are considered generally stable," the Wuhan health authorities said.\nThe infected patients have been quarantined along with 121 people who had been in close contact with them, the statement read.\nAn investigation of experts fro

In [29]:
len(pairs)

62358

In [30]:
# to add to separate file

In [31]:
import torch
import torch.nn as nn

In [32]:
MAX_LENGTH = 600
SOS_token = 0
EOS_token = 1

In [33]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [34]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(in_lang, pair[0])
    target_tensor = tensorFromSentence(out_lang, pair[1])
    return (input_tensor, target_tensor)

In [36]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [37]:
# Train method
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    print("eeeeeeeeeeee",input_length)
    for ei in range(input_length):
#         print("fffffffff",ei)
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [38]:
# Training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
# def trainIters(encoder, decoder, n_iters, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [39]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [40]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
       
        for ei in range(input_length):
           
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [41]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(in_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, out_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

eeeeeeeeeeee 507
eeeeeeeeeeee 348
eeeeeeeeeeee 218
eeeeeeeeeeee 472
eeeeeeeeeeee 415
eeeeeeeeeeee 139
eeeeeeeeeeee 279
eeeeeeeeeeee 381
eeeeeeeeeeee 485


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)